In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import tensorflow as tf
import numpy as np

import time
import random
tf.random.set_seed(3407)
np.random.seed(3407)
random.seed(3407)

from create_spectrogram import (
    create_spectrograms_from_audio_dataset, 
)
from helper_functions import (
    evaluate_prediction,
    get_file_size, 
    convert_bytes, 
    convert_prefetchdataset_to_numpy_arrays,
    predict_and_print_full_results,
    full_int_model_predict,
    get_f1_scores_of_non_overlapping_partitions_full_int_q,
    get_f1_scores_of_bootstarping_partitions_full_int_q,
    )

In [16]:
sample_rate = 16000

In [17]:
train_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/training", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
test_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/testing", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
val_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/validation", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)

label_names = np.array(test_dataset.class_names)
print("Classes: ", label_names)

Found 11292 files belonging to 2 classes.


Found 1393 files belonging to 2 classes.


Found 1380 files belonging to 2 classes.


Classes:  ['non_target' 'target']


In [18]:
train_spectrogram_ds = create_spectrograms_from_audio_dataset(train_dataset, sample_rate = sample_rate).cache().prefetch(tf.data.AUTOTUNE)
test_spectrogram_ds = create_spectrograms_from_audio_dataset(test_dataset, sample_rate = sample_rate).cache().prefetch(tf.data.AUTOTUNE)
val_spectrogram_ds = create_spectrograms_from_audio_dataset(val_dataset, sample_rate = sample_rate).cache().prefetch(tf.data.AUTOTUNE)

x_train_np, y_train_np = convert_prefetchdataset_to_numpy_arrays(train_spectrogram_ds)
x_val_np, y_val_np = convert_prefetchdataset_to_numpy_arrays(val_spectrogram_ds)
x_test_np, y_test_np = convert_prefetchdataset_to_numpy_arrays(test_spectrogram_ds)

In [19]:
BASE_MODEL_FILE_NAME = "spectrogram_models_from_notebooks/cnn/cnn_mel_spec_16kHz_baseline.keras"
baseline_model = tf.keras.models.load_model(BASE_MODEL_FILE_NAME)
baseline_model.summary()
convert_bytes(get_file_size(BASE_MODEL_FILE_NAME), "MB")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 182, 78, 4)        40        
                                                                 
 max_pooling2d (MaxPooling2  (None, 91, 39, 4)         0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 89, 37, 4)         148       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 44, 18, 4)         0         
 g2D)                                                            
                                                                 
 reshape (Reshape)           (None, 3168)              0         
                                                                 
 dense (Dense)               (None, 8)                 2

In [20]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
) = predict_and_print_full_results(baseline_model, x_val_np, y_val_np, model_format="keras")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
) = predict_and_print_full_results(baseline_model, x_test_np, y_test_np, model_format="keras")

Validation dataset:
44/44 [==============================] - 1s 13ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 99.06%
Recall: 99.36%
Precision: 97.91%
F1-score: 98.63%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.986689172446992

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9849647962679836

Test dataset:
44/44 [==============================] - 1s 11ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.99%
Recall: 99.13%
Precision: 97.84%
F1-score: 98.48%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9844262846996378

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9864092038895415


In [21]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(x_val_np).batch(1).take(100):
    yield [input_value]

full_int_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
full_int_converter.optimizations = [tf.lite.Optimize.DEFAULT]
full_int_converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
full_int_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
full_int_converter.inference_input_type = tf.uint8
full_int_converter.inference_output_type = tf.uint8

tflite_model_quant = full_int_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpuv6numy5/assets


INFO:tensorflow:Assets written to: /tmp/tmpuv6numy5/assets
/home/polina/newname/venv/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-07-05 09:36:54.559747: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-05 09:36:54.559796: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-05 09:36:54.559942: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpuv6numy5
2024-07-05 09:36:54.561128: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-05 09:36:54.561142: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpuv6numy5
2024-07-05 09:36:54.563933: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-05 09:36:

In [22]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>


In [23]:
import pathlib

tflite_models_dir = pathlib.Path("spectrogram_models_from_notebooks/cnn")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the quantized model:
tflite_model_quant_file = tflite_models_dir/"cnn_mel_spec_16kHz_full_int_q.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

30616

In [26]:
print("Validation dataset:")
predictions = full_int_model_predict(tflite_model_quant_file, x_val_np)
evaluate_prediction(y_val_np, predictions)

print("\nDevide dataset into 10 non-overlapping patritions and get their mean F1-score")
non_overlap_patritions_f1_scores = get_f1_scores_of_non_overlapping_partitions_full_int_q(tflite_model_quant_file, x_val_np, y_val_np)
print("Non-overlap mean F1-score: ", np.mean(non_overlap_patritions_f1_scores))

print("\nGet 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score")
bootstrap_patritions_f1_scores = get_f1_scores_of_bootstarping_partitions_full_int_q(tflite_model_quant_file, x_val_np, y_val_np)
print("Bootstrap mean F1-score: ", np.mean(bootstrap_patritions_f1_scores))



print("\nTest dataset:")
predictions = full_int_model_predict(tflite_model_quant_file, x_test_np)
evaluate_prediction(y_test_np, predictions)

print("\nDevide dataset into 10 non-overlapping patritions and get their mean F1-score")
non_overlap_patritions_f1_scores = get_f1_scores_of_non_overlapping_partitions_full_int_q(tflite_model_quant_file, x_test_np, y_test_np)
print("Non-overlap mean F1-score: ", np.mean(non_overlap_patritions_f1_scores))

print("\nGet 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score")
bootstrap_patritions_f1_scores = get_f1_scores_of_bootstarping_partitions_full_int_q(tflite_model_quant_file, x_test_np, y_test_np)
print("Bootstrap mean F1-score: ", np.mean(bootstrap_patritions_f1_scores))

print("\nTime for Test dataset:")
time_data = []
for data_point in x_test_np:
    start_time = time.time()
    predictions = full_int_model_predict(tflite_model_quant_file, [data_point])
    elapsed_time = time.time() - start_time
    time_data.append(elapsed_time)
print("\nTime to make a prediction for a single data point")
print(f"Mean: {round(np.mean(time_data), 3)} seconds")
print(f"Max: {round(np.max(time_data), 3)} seconds")
print(f"Min: {round(np.min(time_data), 3)} seconds")

convert_bytes(get_file_size(tflite_model_quant_file), "KB")

Validation dataset:
Accuracy: 99.13%
Recall: 98.94%
Precision: 98.52%
F1-score: 98.73%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9873264585262953

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9862301523372018

Test dataset:
Accuracy: 98.78%
Recall: 98.47%
Precision: 97.83%
F1-score: 98.15%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9813285258051913

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9839190789957197

Time for Test dataset:

Time to make a prediction for a single data point
Mean: 0.001 seconds
Max: 0.004 seconds
Min: 0.001 seconds
File size: 29.898 Kilobytes
